# CNN Image Classification on CIFAR-10

In this notebook, we'll build and train a CNN for image classification using the CIFAR-10 dataset.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

## Load and Explore CIFAR-10 Dataset

In [ ]:
# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Class names
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
               'dog', 'frog', 'horse', 'ship', 'truck']

print(f"Training set shape: {x_train.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Test set shape: {x_test.shape}")
print(f"Test labels shape: {y_test.shape}")
print(f"Number of classes: {len(class_names)}")
print(f"Pixel value range: {x_train.min()} - {x_train.max()}")

In [ ]:
# Visualize sample images
plt.figure(figsize=(15, 10))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.imshow(x_train[i])
    plt.title(f'{class_names[y_train[i][0]]}')
    plt.axis('off')

plt.suptitle('Sample Images from CIFAR-10 Dataset')
plt.tight_layout()
plt.show()

In [ ]:
# Class distribution
unique, counts = np.unique(y_train, return_counts=True)

plt.figure(figsize=(12, 6))
plt.bar([class_names[i] for i in unique], counts)
plt.title('Class Distribution in Training Set')
plt.xlabel('Classes')
plt.ylabel('Number of Samples')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.show()

print("Samples per class:")
for i, count in enumerate(counts):
    print(f"{class_names[i]}: {count}")

## Data Preprocessing

In [ ]:
# Normalize pixel values to [0, 1]
x_train_norm = x_train.astype('float32') / 255.0
x_test_norm = x_test.astype('float32') / 255.0

# Convert labels to categorical (one-hot encoding)
y_train_cat = keras.utils.to_categorical(y_train, 10)
y_test_cat = keras.utils.to_categorical(y_test, 10)

print(f"Normalized pixel range: {x_train_norm.min()} - {x_train_norm.max()}")
print(f"Original label shape: {y_train.shape}")
print(f"Categorical label shape: {y_train_cat.shape}")
print(f"Sample one-hot label: {y_train_cat[0]}")

## Build CNN Model

In [ ]:
def create_cnn_model():
    """Create CNN model for CIFAR-10 classification"""
    
    model = keras.Sequential([
        # First Convolutional Block
        keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(32, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Dropout(0.25),
        
        # Second Convolutional Block
        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Dropout(0.25),
        
        # Third Convolutional Block
        keras.layers.Conv2D(128, (3, 3), activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.25),
        
        # Classifier
        keras.layers.Flatten(),
        keras.layers.Dense(512, activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(10, activation='softmax')
    ])
    
    return model

# Create model
model = create_cnn_model()

# Display model architecture
model.summary()

# Count parameters
total_params = model.count_params()
print(f"\nTotal parameters: {total_params:,}")

In [ ]:
# Visualize model architecture
keras.utils.plot_model(model, show_shapes=True, show_layer_names=True, dpi=150)

## Compile and Train Model

In [ ]:
# Compile model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Define callbacks
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7
    )
]

print("Model compiled successfully!")
print("Callbacks configured:")
print("- Early Stopping (patience=10)")
print("- Learning Rate Reduction (factor=0.5, patience=5)")

In [ ]:
# Data augmentation
datagen = keras.preprocessing.image.ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1
)

# Fit the data generator
datagen.fit(x_train_norm)

# Visualize augmented images
plt.figure(figsize=(15, 5))
sample_image = x_train_norm[0:1]  # Take first image

# Generate augmented versions
augmented_images = []
for batch in datagen.flow(sample_image, batch_size=1):
    augmented_images.append(batch[0])
    if len(augmented_images) >= 5:
        break

# Plot original and augmented images
plt.subplot(1, 6, 1)
plt.imshow(sample_image[0])
plt.title('Original')
plt.axis('off')

for i, aug_img in enumerate(augmented_images):
    plt.subplot(1, 6, i + 2)
    plt.imshow(aug_img)
    plt.title(f'Augmented {i+1}')
    plt.axis('off')

plt.suptitle('Data Augmentation Examples')
plt.tight_layout()
plt.show()

In [ ]:
# Train the model
print("Starting training...")

history = model.fit(
    datagen.flow(x_train_norm, y_train_cat, batch_size=32),
    epochs=50,
    validation_data=(x_test_norm, y_test_cat),
    callbacks=callbacks,
    verbose=1
)

print("Training completed!")

## Visualize Training History

In [ ]:
# Plot training history
def plot_training_history(history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Loss plot
    ax1.plot(history.history['loss'], label='Training Loss', linewidth=2)
    ax1.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
    ax1.set_title('Model Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Accuracy plot
    ax2.plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
    ax2.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
    ax2.set_title('Model Accuracy')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print final metrics
    final_train_acc = history.history['accuracy'][-1]
    final_val_acc = history.history['val_accuracy'][-1]
    final_train_loss = history.history['loss'][-1]
    final_val_loss = history.history['val_loss'][-1]
    
    print(f"Final Training Accuracy: {final_train_acc:.4f}")
    print(f"Final Validation Accuracy: {final_val_acc:.4f}")
    print(f"Final Training Loss: {final_train_loss:.4f}")
    print(f"Final Validation Loss: {final_val_loss:.4f}")

plot_training_history(history)

## Model Evaluation

In [ ]:
# Evaluate on test set
test_loss, test_accuracy = model.evaluate(x_test_norm, y_test_cat, verbose=0)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

# Make predictions
y_pred = model.predict(x_test_norm)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test_cat, axis=1)

# Classification report
print("\nClassification Report:")
print(classification_report(y_true_classes, y_pred_classes, target_names=class_names))

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_true_classes, y_pred_classes)

plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Per-class accuracy
class_accuracy = cm.diagonal() / cm.sum(axis=1)
print("\nPer-class Accuracy:")
for i, acc in enumerate(class_accuracy):
    print(f"{class_names[i]}: {acc:.4f}")

## Visualize Predictions

In [ ]:
# Visualize correct and incorrect predictions
def visualize_predictions(x_test, y_true, y_pred, class_names, num_images=20):
    fig, axes = plt.subplots(4, 5, figsize=(15, 12))
    axes = axes.ravel()
    
    # Get random indices
    indices = np.random.choice(len(x_test), num_images, replace=False)
    
    for i, idx in enumerate(indices):
        # Get prediction
        pred_class = np.argmax(y_pred[idx])
        true_class = y_true[idx]
        confidence = y_pred[idx][pred_class]
        
        # Plot image
        axes[i].imshow(x_test[idx])
        
        # Set title with color coding
        if pred_class == true_class:
            color = 'green'
            title = f'✓ {class_names[pred_class]}\n({confidence:.2f})'
        else:
            color = 'red'
            title = f'✗ {class_names[pred_class]}\nTrue: {class_names[true_class]}\n({confidence:.2f})'
        
        axes[i].set_title(title, color=color, fontsize=10)
        axes[i].axis('off')
    
    plt.suptitle('Model Predictions (Green=Correct, Red=Incorrect)', fontsize=16)
    plt.tight_layout()
    plt.show()

visualize_predictions(x_test, y_true_classes, y_pred, class_names)

In [ ]:
# Visualize most confident correct and incorrect predictions
def show_confident_predictions(x_test, y_true, y_pred, class_names):
    # Get prediction confidences
    pred_classes = np.argmax(y_pred, axis=1)
    confidences = np.max(y_pred, axis=1)
    
    # Correct predictions
    correct_mask = (pred_classes == y_true)
    correct_indices = np.where(correct_mask)[0]
    correct_confidences = confidences[correct_mask]
    
    # Incorrect predictions
    incorrect_mask = (pred_classes != y_true)
    incorrect_indices = np.where(incorrect_mask)[0]
    incorrect_confidences = confidences[incorrect_mask]
    
    # Get most confident correct and incorrect
    most_confident_correct = correct_indices[np.argmax(correct_confidences)]
    most_confident_incorrect = incorrect_indices[np.argmax(incorrect_confidences)]
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Most confident correct
    idx = most_confident_correct
    axes[0].imshow(x_test[idx])
    axes[0].set_title(f'Most Confident Correct\nPredicted: {class_names[pred_classes[idx]]}\nConfidence: {confidences[idx]:.4f}', 
                     color='green')
    axes[0].axis('off')
    
    # Most confident incorrect
    idx = most_confident_incorrect
    axes[1].imshow(x_test[idx])
    axes[1].set_title(f'Most Confident Incorrect\nPredicted: {class_names[pred_classes[idx]]}\nTrue: {class_names[y_true[idx]]}\nConfidence: {confidences[idx]:.4f}', 
                     color='red')
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()

show_confident_predictions(x_test, y_true_classes, y_pred, class_names)

## Feature Visualization

In [ ]:
# Visualize feature maps from first convolutional layer
def visualize_feature_maps(model, x_test, layer_name='conv2d'):
    # Create a model that outputs feature maps
    feature_model = keras.Model(inputs=model.input, 
                               outputs=model.get_layer(layer_name).output)
    
    # Get feature maps for a sample image
    sample_image = x_test[0:1]  # First test image
    feature_maps = feature_model.predict(sample_image)
    
    # Plot original image and feature maps
    fig, axes = plt.subplots(2, 8, figsize=(16, 6))
    
    # Original image
    axes[0, 0].imshow(sample_image[0])
    axes[0, 0].set_title('Original')
    axes[0, 0].axis('off')
    
    # Hide unused subplots in first row
    for i in range(1, 8):
        axes[0, i].axis('off')
    
    # Feature maps (first 8 channels)
    for i in range(8):
        axes[1, i].imshow(feature_maps[0, :, :, i], cmap='viridis')
        axes[1, i].set_title(f'Filter {i+1}')
        axes[1, i].axis('off')
    
    plt.suptitle(f'Feature Maps from {layer_name} Layer')
    plt.tight_layout()
    plt.show()
    
    print(f"Feature maps shape: {feature_maps.shape}")

visualize_feature_maps(model, x_test_norm)

In [ ]:
# Visualize filters from first convolutional layer
def visualize_filters(model, layer_name='conv2d'):
    # Get the weights of the first convolutional layer
    filters = model.get_layer(layer_name).get_weights()[0]
    
    # Normalize filters for visualization
    f_min, f_max = filters.min(), filters.max()
    filters = (filters - f_min) / (f_max - f_min)
    
    # Plot filters
    fig, axes = plt.subplots(4, 8, figsize=(16, 8))
    
    for i in range(32):  # First 32 filters
        row = i // 8
        col = i % 8
        
        # Get filter
        filter_img = filters[:, :, :, i]
        
        axes[row, col].imshow(filter_img)
        axes[row, col].set_title(f'Filter {i+1}')
        axes[row, col].axis('off')
    
    plt.suptitle(f'Learned Filters from {layer_name} Layer')
    plt.tight_layout()
    plt.show()
    
    print(f"Filter shape: {filters.shape}")

visualize_filters(model)

## Model Analysis

In [ ]:
# Analyze model performance by class
def analyze_class_performance(y_true, y_pred, class_names):
    # Calculate per-class metrics
    from sklearn.metrics import precision_recall_fscore_support
    
    precision, recall, f1, support = precision_recall_fscore_support(
        y_true, y_pred, average=None
    )
    
    # Create DataFrame for better visualization
    import pandas as pd
    
    df = pd.DataFrame({
        'Class': class_names,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'Support': support
    })
    
    print("Per-Class Performance:")
    print(df.round(4))
    
    # Visualize metrics
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Precision
    axes[0].bar(class_names, precision)
    axes[0].set_title('Precision by Class')
    axes[0].set_ylabel('Precision')
    axes[0].tick_params(axis='x', rotation=45)
    axes[0].grid(True, alpha=0.3)
    
    # Recall
    axes[1].bar(class_names, recall)
    axes[1].set_title('Recall by Class')
    axes[1].set_ylabel('Recall')
    axes[1].tick_params(axis='x', rotation=45)
    axes[1].grid(True, alpha=0.3)
    
    # F1-Score
    axes[2].bar(class_names, f1)
    axes[2].set_title('F1-Score by Class')
    axes[2].set_ylabel('F1-Score')
    axes[2].tick_params(axis='x', rotation=45)
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return df

performance_df = analyze_class_performance(y_true_classes, y_pred_classes, class_names)

In [ ]:
# Save the trained model
model.save('cifar10_cnn_model.h5')
print("Model saved as 'cifar10_cnn_model.h5'")

# Model summary
print("\n" + "="*50)
print("MODEL SUMMARY")
print("="*50)
print(f"Architecture: Custom CNN")
print(f"Dataset: CIFAR-10")
print(f"Total Parameters: {model.count_params():,}")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Training Time: {len(history.history['loss'])} epochs")
print("="*50)